In [1]:
import sys
sys.path.append("../../../")

In [2]:
import numpy as np

from src.pulsed_power_ml.model_framework.training_data_labelling import trainingdata_switch_detector
from src.pulsed_power_ml.model_framework.data_io import read_training_files
from src.pulsed_power_ml.models.gupta_model.gupta_utils import read_parameters
from src.pulsed_power_ml.model_framework.training_data_labelling import get_switch_features
from src.pulsed_power_ml.model_framework.training_data_labelling import make_labeled_training_data
from src.pulsed_power_ml.model_framework.training_data_labelling import explode_to_complete_label_vector

2023-01-11 17:45:56.019693: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 17:45:56.337245: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-11 17:45:56.337279: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-11 17:45:57.867717: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-1

In [3]:
RAW_DATA_BASE_FOLDER = "../../../../data/raw_data/2022-11-16_training_data/"
TRAINING_DATA_BASE_FOLDER = "../../../../data/training_data/one_class_per_appliance_training_data/"
APPARENT_POWER_BINARY_PREFIX = "FFTApparentPower_"
APPARENT_POWER_BINARY_POSTFIX = "OffOn_FFTSize131072"
PARAMETER_FILE = "../models/gupta_model/parameters_appliance_types.yml"

# Test field

In [4]:
parameter_dict = read_parameters("../models/gupta_model/parameters_appliance_types.yml")

In [5]:
appliance_folder = "r1"
apparent_power_binary = f"{APPARENT_POWER_BINARY_PREFIX}{appliance_folder.upper()}{APPARENT_POWER_BINARY_POSTFIX}"

In [6]:
array_of_data_points = read_training_files(path_to_folder=f"{RAW_DATA_BASE_FOLDER}/{appliance_folder}",
                                           fft_size=2**17)
array_of_apparent_power_spectra = array_of_data_points[:,2*2*16:3*2**16]

In [7]:
switch_positions = trainingdata_switch_detector(spectra=array_of_apparent_power_spectra,
                                                parameters=parameter_dict)

In [8]:
features = get_switch_features(spectra=array_of_apparent_power_spectra,
                               switch_positions=switch_positions,
                               parameters=parameter_dict)

/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [9]:
features, temp_labels = make_labeled_training_data(
    training_file=f"{RAW_DATA_BASE_FOLDER}/{appliance_folder}/{apparent_power_binary}",
    parameter_file="../models/gupta_model/parameters_appliance_types.yml",
    percentage=1.0
)

In [10]:
labels = explode_to_complete_label_vector(
    labels=temp_labels,
    appliance_id=7,
    parameter_file="../models/gupta_model/parameters_appliance_types.yml"
)

In [11]:
labels

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.

# Loop to create complete set of training data

In [12]:
iter_tuple = (
    (1, "e1"),
    (2, "f1"),
    (2, "f2"),
    (3, "fp1"),
    (4, "h1"),
    (4, "h2"),
    (5, "l1"),
    (5, "l2"),
    (6, "ps1"),
    (7, "r1")
)

In [13]:
feature_list = list()
label_list = list()
for percentage in [1.0]:
    for appliance_id, appliance_folder in iter_tuple:
        apparent_power_binary = f"{APPARENT_POWER_BINARY_PREFIX}{appliance_folder.upper()}{APPARENT_POWER_BINARY_POSTFIX}"
        features, temp_labels = make_labeled_training_data(
            training_file=f"{RAW_DATA_BASE_FOLDER}/{appliance_folder}/{apparent_power_binary}",
            parameter_file=PARAMETER_FILE,
            percentage=percentage
        )

        labels = explode_to_complete_label_vector(
            labels=temp_labels,
            appliance_id=appliance_id,
            parameter_file=PARAMETER_FILE
        )

        feature_list.append(features)
        label_list.append(labels)

/home/thomas/projects/fair/call_3/repository/src/pulsed_power_ml/notebooks/../../../src/pulsed_power_ml/model_framework/data_io.py:40: UserWarning: Length of array in file ../../../../data/raw_data/2022-11-16_training_data//l2/FFTApparentPower_L2OffOn_FFTSize131072 is not a multiple of FFT Size (131072)!
Ignoring last 81915 entries to continue.
  warnings.warn(f"Length of array in file {path_to_file} is not a multiple of FFT Size ({fft_size})!\n"


In [14]:
all_features = np.vstack(feature_list)
all_labels = np.vstack(label_list)

In [15]:
np.savetxt(f"{TRAINING_DATA_BASE_FOLDER}/100_0_split/features.csv", all_features, delimiter=",")
np.savetxt(f"{TRAINING_DATA_BASE_FOLDER}/100_0_split/labels.csv", all_labels, delimiter=",")